## Setup and support functions

In [1]:
import numpy as np
import time
from typing import List, Generator, Tuple

def service_time_with_no_shows(s: List[float], q: float) -> List[float]:
    """
    Adjust a distribution of service times for no-shows.

    The adjusted service time distribution accounts for the probability q of no-shows.

    Parameters:
    s (List[float]): The original service time probability distribution.
    q (float): The no-show probability.

    Returns:
    List[float]: The adjusted service time distribution.
    """
    # Adjust the service times by multiplying with (1 - q)
    s_adj = [(1 - q) * float(si) for si in s]  # JB: why the float conversion?
    # Add the no-show probability to the zero service time  # JB: In the paper you say, you start with index 1, no?
    s_adj[0] = s_adj[0] + q  # JB: s_adj[0] += q
    return s_adj

def compute_convolutions(probabilities, N, q=0.0):  # JB: I think hint typing everywhere is a good practice
    """
    Computes the k-fold convolution of a probability mass function for k in range 1 to N.  # JB: Introduce PMF here.

    Parameters:
    probabilities (list of floats): The PMF represented as a list where the index is the service time and the value is the probability. JB: Note that this function is generic and does not have to be used for service times only.
    N (int): The maximum number of convolutions to compute.

    Returns:
    dict: A dictionary where keys are k and values are the convoluted service times (PMFs).
    """
    convolutions = {}
    result = probabilities.copy()  # JB: I do not think it is necessary to copy probabilities: service_time_with_no_show() does not change it inplace.
    result = service_time_with_no_shows(result, q)  # is result a meaningful name?
    for k in range(1, N + 1):
        if k == 1:
            convolutions[k] = result
        else:
            result = np.convolve(result, probabilities)  # JB: shouldn't you work with service_time_with_no_shows(probabilities, q) instead of probabilities? 
            convolutions[k] = result
    """
    JB: I think starting from line 35, something like this is easier (asumming you have to work with probabilities, although I doubt that):
    
    convolutions = {1: service_time_with_no_shows(probabilities, q)}
    for k in range(2, N + 1):
        convolutions[k] = np.convolve(convolutions[k - 1], probabilities)
    return convolutions
    
    Furthermore, ChatGPT hinted on the following: "For large N, consider using algorithms designed for efficient repeated convolutions (e.g., FFT-based methods)." I do not think these convolutions are the bottleneck per se, but it is something to keep in mind.    
    """
    return convolutions

def test_compute_convolutions():  # JB: Good that you include tests! Test it also for (small) examples for which you know the convolution result, either found on internet, hand-computed, or (trivial) edge cases.
    # Define a sample probability mass function
    # probabilities = [0.1, 0.2, 0.3, 0.4]  # Service times 0, 1, 2, 3
    probabilities = [0., 0.00599585, 0.00644478, 0.02427306, 0.04753406, 0.07355202, 0.16864656, 0.12338866, 0.10182152, 0.09778246, 0.07283463, 0.05794873, 0.05378825, 0.04945164, 0.04805953, 0.02797309, 0.02179657, 0.0183278, 0.0003808 ]
    N = 20
    convolutions = compute_convolutions(probabilities, N, q=0.0)

    # Test that the probabilities sum to 1 for each convolution
    for k in range(1, N + 1):
        conv_pmf = convolutions[k]
        total_prob = sum(conv_pmf)
        assert abs(total_prob - 1.0) < 1e-6, f"Total probability at k={k} does not sum to 1."
        print(f"Convolution at k={k}: {conv_pmf}")

    print("All tests passed.")

# Run the test function
test_compute_convolutions()

Convolution at k=1: [0.0, 0.00599585, 0.00644478, 0.02427306, 0.04753406, 0.07355202, 0.16864656, 0.12338866, 0.10182152, 0.09778246, 0.07283463, 0.05794873, 0.05378825, 0.04945164, 0.04805953, 0.02797309, 0.02179657, 0.0183278, 0.0003808]
Convolution at k=2: [0.00000000e+00 0.00000000e+00 3.59502172e-05 7.72838683e-05
 3.32610443e-04 8.82883251e-04 2.08388832e-03 5.27800631e-03
 9.48357180e-03 1.79910271e-02 2.99178643e-02 4.36157419e-02
 6.26532822e-02 7.08202826e-02 7.49764079e-02 7.81568742e-02
 7.60583714e-02 7.30011459e-02 7.05386938e-02 6.70262150e-02
 6.30866822e-02 5.52932979e-02 4.76385973e-02 4.01495458e-02
 3.01858929e-02 2.30535773e-02 1.76944630e-02 1.30329276e-02
 9.60076381e-03 6.86027371e-03 4.73120425e-03 3.01874811e-03
 1.53706300e-03 8.20270657e-04 3.52508521e-04 1.39584525e-05
 1.45008640e-07]
Convolution at k=3: [0.00000000e+00 0.00000000e+00 0.00000000e+00 2.15552110e-07
 6.95073723e-07 3.36498163e-06 1.10220124e-05 3.25759705e-05
 9.40642038e-05 2.25360713e-04 5

## Tests for functions for calculating objective values

The difference between `calculate_objective_serv_time_lookup` and `calculate_objective` is that the former looks up pre-calculated convolutions of service times (adjusted for no-shows) from a numpy array, while the latter must calculates the convolutions for every single interval seperately.

The test verifies whether both functions return the same results and measure the difference in execution times.

In [10]:
def calculate_objective_serv_time_lookup(schedule: List[int], d: int, q: float, convolutions: dict) -> Tuple[float, float]:
    """
    Calculate the objective value based on the given schedule and parameters using precomputed convolutions.

    This function uses precomputed convolutions of the service time distribution,
    starting from the 1-fold convolution (key 1) which contains the adjusted service time distribution.

    Parameters:
    schedule (List[int]): A list representing the number of patients scheduled in each time slot.
    d (int): Duration threshold or maximum allowed service time per slot.
    q (float): No-show probability.
    convolutions (dict): Precomputed convolutions of the service time distribution, with key 1 containing the adjusted service time distribution.

    Returns:
    Tuple[float, float]:
        - ewt (float): The sum of expected waiting times.
        - esp (float): The expected spillover time (overtime).
    """
    sp = np.array([1], dtype=np.float64)  # Initial service process (no waiting time) JB: What does this represent?
    ewt = 0  # Total expected waiting time

    for x in schedule:
        if x == 0:
            # No patients in this time slot
            # Adjust sp for the duration d (service process moves ahead)
            sp_new = []
            sp_new.append(np.sum(sp[:d + 1]))  # JB: replace this and previous line by: sp_new = [np.sum(sp[:d + 1])]
            sp_new.extend(sp[d + 1:])
            sp = np.array(sp_new)  
            
            """
            JB: perhaps the above is faster using numpy only:
            
            sp = np.concatenate([np.sum(sp[:d + 1]), sp[d + 1:]])
            
            or something like this is even faster:
            
            sp[d] = np.sum(sp[:d + 1])
            sp = sp[d:]
            
            the test at the end of this notebook seems to suggest that the last approach is the fastest.
            
            """
        else:
            # Patients are scheduled in this time slot
            wt_temp = [sp.copy()]  # JB: The switch from st to wt here is a bit confusing to me.
            # Add expected waiting time for the first patient
            ewt += np.dot(range(len(sp)), sp)
            # For each additional patient
            for i in range(1, x):
                # The waiting time distribution for the ith patient is the convolution
                # of the previous patient's waiting time with s (adjusted service time distribution)
                conv_s = convolutions.get(1)  # Adjusted service time distribution  JB: If I am not mistaken, you only get the first convolution result, and not all the others?
                wt = np.convolve(wt_temp[i - 1], conv_s)
                wt_temp.append(wt)
                ewt += np.dot(range(len(wt)), wt)  # JB: I am wondering if this is needed, based upon the ewt of the first patient in interval t, the expectations of the other patients in the interval is easy right?
            # Update sp for the next time slot
            conv_s = convolutions.get(1)  # Adjusted service time distribution
            sp = np.convolve(wt_temp[-1], conv_s)
            # Adjust sp for duration d  JB: See also the above. Furthermore, I think a dedicated function would be useful that calculates max(X-d, 0) for a random variable X with given PMF p and integer d.
            sp_new = []
            sp_new.append(np.sum(sp[:d + 1]))
            sp_new.extend(sp[d + 1:])
            sp = np.array(sp_new)
    # Expected spillover time
    esp = np.dot(range(len(sp)), sp)
    return ewt, esp

def calculate_objective(schedule: List[int], s: List[float], d: int, q: float) -> Tuple[float, float]:
    """
    Calculate the objective value based on the given schedule and parameters.

    This function adjusts the service times distribution for no-shows, calculates
    the waiting times for all patients in the schedule, sums the expected
    waiting times, and calculates the spillover time for the last interval (overtime).

    Parameters:
    schedule (List[int]): A list representing the number of patients scheduled in each time slot.
    s (List[float]): Service times probability distribution.
    d (int): Duration threshold or maximum allowed service time per slot.
    q (float): No-show probability.

    Returns:
    Tuple[float, float]:
        - ewt (float): The sum of expected waiting times.
        - esp (float): The expected spillover time (overtime).
    """
    # Adjust the service time distribution for no-shows
    s = service_time_with_no_shows(s, q)
    # Initialize the service process (probability distribution of waiting times)
    sp = np.array([1], dtype=np.float64)
    wt_list = []
    ewt = 0  # Expected waiting time
    for x in schedule:
        if x == 0:
            # No patients in this time slot
            wt_temp = [np.array(sp)]
            wt_list.append([])
            sp = []
            sp.append(np.sum(wt_temp[-1][:d + 1]))
            sp.extend(wt_temp[-1][d + 1:])
        else:
            # Patients are scheduled in this time slot
            wt_temp = [np.array(sp)]
            # Add expected waiting time for the first patient
            ewt += np.dot(range(len(sp)), sp)
            # For each additional patient
            for i in range(x - 1):
                # Convolve the waiting time with the service time distribution
                wt = np.convolve(wt_temp[i], s)
                wt_temp.append(wt)
                # Add expected waiting time
                ewt += np.dot(range(len(wt)), wt)
            wt_list.append(wt_temp)
            # Update the service process for the next time slot
            sp = []
            convolved = np.convolve(wt_temp[-1], s)
            sp.append(np.sum(convolved[:d + 1]))
            sp.extend(convolved[d + 1:])
        # Calculate expected spillover time
        esp = np.dot(range(len(sp)), sp)
    return ewt, esp

# JB: I highly recommend to also just write a discrete event simulation to compare the results also with that. Didn't you do that already? It is a good way to verify the correctness of the code. 

def test_calculate_objective_serv_time_lookup_with_timing():
    """
    Test the calculate_objective_serv_time_lookup function by comparing its outputs
    and execution time with the original calculate_objective function.

    The test verifies that both functions produce the same expected waiting time (ewt)
    and expected spillover time (esp) for a sample schedule and parameters, and compares
    their execution times.
    """

    # Sample service time distribution (probabilities)
    s = [0., 0.00599585, 0.00644478, 0.02427306, 0.04753406, 0.07355202, 0.16864656, 0.12338866, 0.10182152, 0.09778246, 0.07283463, 0.05794873, 0.05378825, 0.04945164, 0.04805953, 0.02797309, 0.02179657, 0.0183278, 0.0003808 ]

    # Normalize the service time distribution to sum to 1
    total_prob = sum(s)
    s = [prob / total_prob for prob in s]

    # Sample schedule: Number of patients scheduled in each time slot
    schedule = [12, 2, 3, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]

    # Parameters
    d = 5       # Duration threshold or maximum allowed service time per slot
    q = 0.1      # No-show probability

    # Precompute convolutions for the maximum number of patients in any time slot
    N = max(schedule)
    convolutions = compute_convolutions(s, N, q)

    # Measure execution time for the original function
    start_time_original = time.time()
    ewt_original, esp_original = calculate_objective(schedule, s, d, q)
    end_time_original = time.time()
    time_original = end_time_original - start_time_original

    # Measure execution time for the lookup function
    start_time_lookup = time.time()
    ewt_lookup, esp_lookup = calculate_objective_serv_time_lookup(schedule, d, q, convolutions)
    end_time_lookup = time.time()
    time_lookup = end_time_lookup - start_time_lookup

    # Define a tolerance for floating-point comparisons
    tolerance = 1e-6

    # Check if the expected waiting times are approximately equal
    ewt_match = abs(ewt_original - ewt_lookup) < tolerance
    # Check if the expected spillover times are approximately equal
    esp_match = abs(esp_original - esp_lookup) < tolerance

    # Print the results
    print("Original Function Output:")
    print(f"Expected Waiting Time (ewt): {ewt_original}")
    print(f"Expected Spillover Time (esp): {esp_original}")
    print(f"Execution Time: {time_original:.6f} seconds\n")

    print("Lookup Function Output:")
    print(f"Expected Waiting Time (ewt): {ewt_lookup}")
    print(f"Expected Spillover Time (esp): {esp_lookup}")
    print(f"Execution Time: {time_lookup:.6f} seconds\n")

    # Assert that both outputs match within the defined tolerance
    assert ewt_match, "Expected Waiting Times do not match."
    assert esp_match, "Expected Spillover Times do not match."

    print("Test passed: The outputs of both functions match within the defined tolerance.")

    # Compare execution times
    if time_lookup > 0:
        speedup = time_original / time_lookup
        print(f"The lookup function is {speedup:.2f} times faster than the original function.")
    else:
        print("Execution time for the lookup function is too small to measure speedup.")

# Run the test function
test_calculate_objective_serv_time_lookup_with_timing()

Original Function Output:
Expected Waiting Time (ewt): 1336.3335724956953
Expected Spillover Time (esp): 65.40449287413372
Execution Time: 0.003008 seconds

Lookup Function Output:
Expected Waiting Time (ewt): 1336.3335724956953
Expected Spillover Time (esp): 65.40449287413372
Execution Time: 0.000993 seconds

Test passed: The outputs of both functions match within the defined tolerance.
The lookup function is 3.03 times faster than the original function.


In [9]:
# JB: Test to compare the speed comparison of the three approaches to update sp

import time

# Generate random data for testing
np.random.seed(42)
sp = np.random.rand(10**6)
d = 100  # Partition index

# Approach 1: Using list
start_1 = time.time()
sp_new = []
sp_new.append(np.sum(sp[:d + 1]))
sp_new.extend(sp[d + 1:])
sp_1 = np.array(sp_new)
time_1 = time.time() - start_1

# Approach 2: Using np.concatenate
start_2 = time.time()
sp_2 = np.concatenate([[np.sum(sp[:d + 1])], sp[d + 1:]])
time_2 = time.time() - start_2

# Approach 3: In-place modification
sp_copy = sp.copy()  # To avoid altering the original
start_3 = time.time()
sp_copy[d] = np.sum(sp_copy[:d + 1])
sp_3 = sp_copy[d:]
time_3 = time.time() - start_3

# Verify if all results are the same
are_equal = np.array_equal(sp_1, sp_2) and np.array_equal(sp_2, sp_3)

time_1, time_2, time_3, are_equal

(0.06954550743103027, 0.0029904842376708984, 0.0009996891021728516, True)